<a href="https://colab.research.google.com/github/Adrian-Muino/DMML2022_Geneva/blob/main/Colab_Notebooks/3.DMML_2022_Geneva_Embeding_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A. Introduction

In the previous colab we put our focus on choosing the best model in order to reach the best accuracy. In this colab, we choose to manipulate the data in order to improve the accuracy. We decided to use word embeddings (see definition below) to improve our accuracy. To simplify the lecture of the collab we kept only the best model: Logistic regression.




<p align="center">
<img width="800" src="https://user-images.githubusercontent.com/114933900/208532349-5a3facf6-3931-45d1-8df0-23612263a568.jpg">
<p>

<justify> Word embeddings are a way to represent words as numerical vectors. The idea behind word embeddings is to capture the meaning of a word in a continuous, low-dimensional space, such that semantically similar words are close to each other in that space.

There are different ways to create word embeddings, but a common approach is to use a neural network to learn the embeddings from large amounts of text data. The neural network takes as input a one-hot encoded representation of a word, where each word in the vocabulary is represented by a unique vector with all zeros except for a 1 in the index corresponding to that word. The network then learns to map this one-hot encoded vector to a continuous, low-dimensional space through an embedding layer.

Once the network has learned the embeddings, they can be used as input to other machine learning models, such as natural language processing (NLP) tasks like text classification or language translation. The advantage of using word embeddings is that they capture the meaning of words in a way that is more meaningful for machine learning models than a one-hot encoded representation, which treats each word as an independent entity with no relationship to other words.</justify>

>[A. Introduction](#scrollTo=fTgNKQBs-Ibn)

>[B. Prerequisites](#scrollTo=FVWMhA2w_Hoa)

>>[Installations](#scrollTo=rBePMgY0_JiI)

>>[Imports](#scrollTo=j4brENxg_NpV)

>[C. SentenceTransformer Model](#scrollTo=A_f9Adr6_YJM)

>[D. Some improvement in the submission](#scrollTo=KT--zESy_n41)



#B. Prerequisites

##Installations

In [ ]:
!pip install sentence-transformers
!python -m spacy download fr_core_news_sm
!python -m spacy link fr_core_news_sm fr
!pip install tensorflow_hub
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-21 14:37:24.415093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 14:37:27.717456: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-21 14:37:27.717824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could no

## Imports

In [ ]:
# Imports the functions we use all along our projects that are in python file in our GitHub
import requests
url = 'https://raw.githubusercontent.com/Adrian-Muino/DMML2022_Geneva/main/Colab_Notebooks/dmml_2022_geneva_functions.py'

r = requests.get(url)

with open('dmml_2022_geneva_functions.py', 'w') as f:
    f.write(r.text)

In [ ]:
import string
import re
import pandas as pd
import spacy
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from dmml_2022_geneva_functions import *
import matplotlib.pyplot as plt

import seaborn as sns


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


#C. SentenceTransformer Model

SentenceTransformer("distiluse-base-multilingual-cased-v1") is a pre-trained model for natural language processing (NLP) tasks, specifically for generating sentence embeddings. Sentence embeddings are numerical representations of sentences or short texts that capture the meaning of the sentence in a continuous, low-dimensional space.

The SentenceTransformer model is trained to generate sentence embeddings by taking a sentence as input and producing a fixed-length vector as output. The model is based on the "distilBERT" architecture, which is a smaller, faster version of the BERT model developed by Google. BERT (Bidirectional Encoder Representations from Transformers) is a transformer-based architecture that has been successful in a wide range of NLP tasks.

The "distiluse-base-multilingual-cased-v1" version of the model is trained on a large dataset of texts in multiple languages and is able to handle a variety of languages and writing systems. It is also cased, meaning that it is able to distinguish between upper and lower case letters.

Once the SentenceTransformer model is loaded, you can use it to generate sentence embeddings by passing a sentence or list of sentences to the encode method. You can then use these embeddings as input to other machine learning models or for a variety of NLP tasks, such as text classification or text similarity.

https://www.sbert.net/docs/pretrained_models.html


In [ ]:
# load the data from our github repository
training_data = 'https://raw.githubusercontent.com/Adrian-Muino/DMML2022_Geneva/main/Data/training_data.csv'
unlabelled_data = 'https://raw.githubusercontent.com/Adrian-Muino/DMML2022_Geneva/main/Data/unlabelled_test_data.csv'

df = df_train = pd.read_csv(training_data)
df_unlabelled = pd.read_csv(unlabelled_data)

Tokenization is the process of dividing a piece of text into individual tokens, which are usually defined as words or punctuation marks. Tokenization is an important step in many natural language processing (NLP) tasks, as it allows you to work with individual words and punctuation marks rather than the raw text.

One reason to tokenize text before generating word embeddings is that word embeddings are typically generated for individual words, rather than for longer pieces of text like sentences or paragraphs. By tokenizing the text into words, you can generate word embeddings for each word, which can then be used as input to other machine learning models or for tasks such as text classification or language translation.

Another reason to tokenize text before generating word embeddings is that it allows you to control the granularity of the embeddings. For example, you might want to generate embeddings for individual words rather than longer phrases in order to capture the meaning of each word more accurately. Tokenization also allows you to exclude punctuation marks or other types of non-content words (like stop words) from the embeddings, which can be useful in some cases.

Overall, tokenization is an important step in many NLP pipelines and is often performed before generating word embeddings or performing other tasks with the text.

In [ ]:
model= SentenceTransformer("distiluse-base-multilingual-cased-v1")
df["tokenize"]= df["sentence"].apply(spacy_tokenizer_2)
df["embeddings"]=df["tokenize"].apply(model.encode)

In [ ]:
X = df["embeddings"].to_list()
y =df["difficulty"].to_list()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
LR = LogisticRegression(solver = "saga", multi_class = 'multinomial')
LR.fit(X_train,y_train)

LogisticRegression(multi_class='multinomial', solver='saga')

In [ ]:
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.5302083333333333


#D. Some improvement in the submission

In [ ]:
# Used the best model in terms of accuracy
df_unlabelled["tokenize"]= df_unlabelled["sentence"].apply(spacy_tokenizer_2)
df_unlabelled["embeddings"]=df_unlabelled["tokenize"].apply(model.encode)

X = df_unlabelled["embeddings"].to_list()

geneva_predictions_LR_embeddings = LR.predict(X)

fourth_submission = pd.DataFrame(geneva_predictions_LR_embeddings ,columns=["difficulty"])
fourth_submission = fourth_submission.rename_axis("id")

fourth_submission.to_csv("Geneva_predictions_LR_embeddings.csv") 

# Conclusion

In conclusion, using logistic regression for embedding allows for the representation of categorical variables in a numerical format, which can then be input into a machine learning model. This can be especially useful when dealing with large categorical variables with many categories, as it allows for the efficient incorporation of this information into the model. However, it is important to carefully consider the choice of embedding method and ensure that it is appropriate for the task at hand. Additionally, it may be necessary to tune the hyperparameters of the logistic regression model in order to achieve the best performance. Overall, using logistic regression for embedding can be a powerful tool for improving the predictive performance of machine learning models.

We would like to point out that the text analysis models are much more efficient for English texts than for French texts. Despite the fact that the models we use are trained on French texts.

We think that models are better in english than in french for several reason:

One reason could be the availability of high-quality labeled training data. Pretrained models are typically trained on large datasets of labeled text, and it is likely that there is a larger amount of labeled English text available for training compared to French text. This could lead to pretrained English models having better performance due to their larger and more diverse training data.

another reason could be the complexity of the language itself. English is a relatively simple and regular language compared to many other languages, which may make it easier for machine learning models to learn and generalize to new examples. French, on the other hand, has more complex grammar and syntax, and may be more difficult for machine learning models to learn effectively.

Finally, the cultural and societal context in which the language is used can also play a role in the performance of text classification models. For example, if the pretrained model was trained on a dataset that is primarily focused on English-speaking cultures, it may not be as effective at classifying text from other cultures, such as French-speaking cultures.

